In [ ]:
import pandas as pd
import pickle

# 读取 CSV 文件
df = pd.read_csv('data/datashare.csv')


# 输出基本信息（包括列数据类型、非空数量等）
print(df.info())

# 显示前几行数据，便于了解数据结构
print(df.head())



ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'